In [1]:
#importing things
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
%matplotlib inline
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.metrics import confusion_matrix
from scipy.special import expit
from sklearn.metrics import recall_score, precision_score, f1_score
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_score

In [2]:
##reading files
rider_provider='bank-marketing.csv'
data=pd.read_csv(rider_provider)

In [3]:
data.head()

,age,job,salary,marital,education,targeted,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,response
0,58,management,100000,married,tertiary,yes,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,60000,single,secondary,yes,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,120000,married,secondary,yes,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,20000,married,unknown,no,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,0,single,unknown,no,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [5]:
data.shape

(45211, 19)

### 1.Describe the pdays column, make note of the mean, median and minimum values. Anything fishy in the values?

In [ ]:
data.info()

In [ ]:
data.pdays.describe()

 #THE VALUES FOR 25,50 AND 75 percentiles ARE NEGATIVE AND THE PDAYS VALUES ARE NEGATIVE WHICH SEEMS FISHY HERE.

In [ ]:
#Describe the pdays column again, this time limiting yourself to the relevant values of pdays. How different are the mean and the median values?
data.pdays.value_counts()

In [ ]:
#WE CAN CONSIDER REMOVING THE NEGATIVE VALUES AS IT IS RESEMBLING THE MISSING RECORDS. 
#AS MANY VALUES ARE MISSING WE CANT GO FOR IT NOW

In [ ]:
data.pdays[data.pdays>0].describe()

In [ ]:
#removing negative values
data.drop(data[data.pdays < 0].index, inplace=True)


In [ ]:
# horizontal bar chart for median values of education and balance relation
sns.set()
sns.barplot(x='education', y="balance", data=data, estimator=np.median)


In [ ]:
print("thus the heighest median value is for the customers whose education details are not known")

In [ ]:
#plotting box plot
sns.set()
sns.boxplot(data.pdays)
plt.title("PDAYS")
plt.figure(figsize=(20,10))
plt.show()


In [ ]:
data.pdays.describe()

In [ ]:
print("YES! I CAN SEE FEW OUTLIERS HERE IN THE pdays COLUMNS ")

In [ ]:
#NOW AS OUTLIERS ARE PRESENT I CAN NOT TRAIN WITH THIS SET THUS REMOVING OUTLIERS
data.drop(data[data.pdays > 610].index, inplace=True)
#plotting box plot
sns.set()
sns.boxplot(data.pdays)
plt.title("PDAYS")
plt.figure(figsize=(20,10))
plt.show()

In [ ]:
print("removed outliers and thus ready to split and train")

In [ ]:
data.pdays.describe()

In [ ]:
data.mode()

In [ ]:
#performing EDA
data.job.value_counts().plot(kind="bar")


In [ ]:
data.marital.value_counts().plot(kind="bar")

In [ ]:
data.education.value_counts().plot(kind="bar")

In [ ]:
data.response.value_counts().plot(kind="bar")

In [ ]:
sns.pairplot(data = data)

In [ ]:
#identifying the multi-colinearity
plt.figure(figsize = (10,10))
sns.heatmap(data.corr(), cmap = 'coolwarm', annot = True)

##ConvertIng the response variable to a convenient form

In [ ]:
#replacing the yeses with 1 and no with 0 in the review section as response are our required section for training
data.response.replace(('yes','no'),(1,0),inplace=True)

In [ ]:
data.response.head()

In [ ]:
data.head()

### Are the features about the previous campaign data useful

yes they are usefull as the responces says the engagement and sucess of the compaign

## Are pdays and poutcome associated with the target?

I DONT THINK SO AS PREVIOUS DATA WERE MISSING AND NO CONSISTANT RELATION WAS OBSERBED FROM THE DATASET AND THE MOSTLY RESPONSES WERE UNKNOWN THUS I DONT TAKE IT INTO MY CONSIDERATION

#performing logistic regression

In [ ]:
numeric = ['age','balance', 'day', 'duration','campaign', 'pdays', 'previous']
categorical = ['job','marital','education','default','housing','loan','contact','month', 'poutcome']

In [ ]:

scalar = StandardScaler()
scalar.fit(data[numeric])
data[numeric] = scalar.transform(data[numeric])
data

In [ ]:
data = pd.get_dummies(data, columns=categorical,drop_first=True)   

data


In [ ]:
data.columns

In [ ]:
X = data[['age', 'balance', 'duration', 'campaign', 'pdays', 'previous',
        'job_blue-collar', 'job_housemaid',
       'job_management', 'job_retired', 'job_services',
       'job_student', 'job_technician', 'job_unemployed',
       'marital_married', 'marital_single', 'education_secondary',
       'education_tertiary', 'education_unknown', 'housing_yes',
       'loan_yes', 'contact_telephone', 'contact_unknown',
       'month_dec', 'month_feb', 'month_jan', 'month_jun',
       'month_mar', 'month_may', 'month_oct', 'month_sep',
       'poutcome_other', 'poutcome_success', 'poutcome_unknown'] ]

y = data["response"]

In [ ]:
OLS= sm.OLS(y,X).fit()
OLS.summary()

__'default_yes' , 'job_unknown', 'month_nov', 'month_aug', , 'job_entrepreneur' , 'job_self-employed', 'month_jul' , 'day' as been removed as the p-value of the variables are greater than 0.05 which means they are not significant

LOGISTIC REGRESSION

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=33)

In [ ]:
from sklearn.linear_model import LogisticRegression
lgr = LogisticRegression()
model = lgr.fit(X_train,np.array(y_train))
print(model)

In [ ]:
y_pred = lgr.predict(X_test)
print(y_pred)

In [ ]:

cvs = cross_val_score(model, X_train, y_train, cv=5)
np.mean(cvs)

### USING RFETHUS THE ACCURECY OF THE MODE IS APPROX 84.08%

In [ ]:

print("Classification Report:\n ", classification_report(y_test, y_pred))

### USING RFE

In [ ]:

rfe = RFE(lgr, 15)
rfe.fit(X_train,y_train)

In [ ]:

rfe = RFE(lgr, 15)
rfe.fit(X_train,y_train)

In [ ]:
print(X_train.columns[rfe.support_])
cols = X_train.columns[rfe.support_]
lgr.fit(X_train[cols],y_train)
y_pred2 = lgr.predict(X_test[cols])

### FEATURE IMPORTANCE

In [ ]:

model = SelectFromModel(lgr)
model.fit(X_train,y_train)
features = model.get_support()
feature_name = X_train.columns[features]
feature_name

In [ ]:
importances=feature_name
feature_importances=pd.Series(importances).sort_values(ascending=False)
sns.barplot(x=feature_importances[0:10], y=feature_importances.index[0:10])
sns.set(rc={'figure.figsize':(35,35)})
plt.title('Feature Importance',size=20)
plt.ylabel("Features")
plt.show()

## RANDOM FOREST

In [ ]:

rf = RandomForestClassifier(n_estimators=45,random_state=0,max_depth=30)
rf = rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
print(y_pred)

In [ ]:

cvs = cross_val_score(rf, X_train, y_train, cv=5)
np.mean(cvs)

### THE ACCURACY OF THE MODEL IS 84.8% APPROX

In [ ]:
print("Classification Report:\n ", classification_report(y_test, y_pred))

SINCE THE ACCURACY OF RANDOM FOREST IS SLIGHTLY MORE THAN THE ACCURACY OF LOGISTIC I WILL GO FOR RANDOM FOREST.
AND ALSO F1 SCORE INCREASES IN  THE CASE OF RANDOM FOREST WHICH IS ALSO A RESON FOR ME TO GOING WITH THE RANDOM FOREST.